In [4]:
import sys
from pathlib import Path

# For Jupyter Notebook, use the current working directory instead of __file__
sys.path.insert(0, str(Path.cwd().parent))

from colocation.synthetic import GeneratorParams, SyntheticSpatialGenerator
from colocation.miner import CoLocationMiner

In [6]:
params = GeneratorParams(
    P=10,        # giảm từ 20
    I=200,       # giảm từ 500
    D=5000.0,    # giảm từ 10000.0
    F=10,        # giảm từ 20
    Q=3,         # giảm từ 5
    m=10000,     # giảm từ 50000
    min_dist=50.0,
    clumpy=1,
)

gen = SyntheticSpatialGenerator(params, seed=42)
ds = gen.generate()

miner = CoLocationMiner(dataset=ds,
                        min_dist=params.min_dist,
                        min_prev=0.2)

cliques_ids, prev_ids = miner.run_ids()
cliques_nds, prev_nds = miner.run_nds()

print("Số instance:", len(ds.instances))

Số instance: 10000


In [7]:
print("Số clique IDS:", len(cliques_ids))


Số clique IDS: 42341


In [8]:
print("Số clique NDS:", len(cliques_nds))


Số clique NDS: 26810


In [9]:
print("Số prevalent pattern (IDS):", len(prev_ids))


Số prevalent pattern (IDS): 53


In [10]:
print("Số prevalent pattern (NDS):", len(prev_nds))

Số prevalent pattern (NDS): 53
